<img src="https://drive.google.com/uc?id=1-hPP-XPm9_5M3orUgmompcVleQ5xvPST" style="Width:1000px">

# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import SGDClassifier, SGDRegressor, LinearRegression, LogisticRegression, LogisticRegressionCV, Lasso, Ridge
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, precision_score, f1_score, r2_score, roc_auc_score, mean_absolute_error
import warnings
import scipy
warnings.filterwarnings('ignore')
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, precision_score, f1_score, r2_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder

🎯 The `ML_Houses_dataset.csv` is a classic made up of 84 features! The data comes from the Boston housing market. Here, you can practice doing the following:

- Investigate the features
- Preprocess and encode them accordingly
- Brainstorm feature engineering
- Add them to your modelling
- Perform feature selection
- Train and test models
- Enter your best performance into the <a href="https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques">Kaggle House Price Competition</a> to measure yourselve against other data scientists (make sure to read the rules: they ask for some peculiar metrics, and you may need to download the data straight from them if you want to enter the competitions)

ℹ️ Make sure you refer to the description of the dataset [available here](https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_Houses_dataset_description.txt). This exercise has no test and no set solutions, but you can find many good example solutions on Kaggle or elsewhere. Plus you can compete against champions - good luck!

In [2]:
# from nbta.utils import download_data
# download_data(id='144IUbRiGahdHkXgxJcZFsYQ80NJxJMiR')

In [3]:
!ls raw_data

houses.csv  ML_Houses_dataset.csv  ML_Houses_dataset_description.txt


# helpers

In [4]:
 # bin

def split_data(df, target=None):
    
    if target == None:
        target = list(df.columns)[-1]
    return df.drop(columns=[target], axis=1), df[[target]]

def nan_ratio_col(df, col):
    nan_ratio = sum(df[col].isna()) / len(df[col])
    return nan_ratio

# Load data

In [5]:
ml_df = pd.read_csv("raw_data/ML_Houses_dataset.csv")
houses_df = pd.read_csv("raw_data/houses.csv")

In [6]:
kaggle_test_df = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")
kaggle_train_df = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")


In [7]:
# check both dfs have the same cols
assert list(ml_df.columns) == list(houses_df.columns)

In [8]:
len(ml_df.columns)

85

In [9]:
houses_df.shape

(1760, 85)

In [10]:
print(len(ml_df))
ml_df.drop_duplicates()
print(len(ml_df))


1760
1760


In [11]:
# main data
X, y = split_data(ml_df)

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42)

In [12]:
# kaggle data: get X_train, y_train, X_test

X_train, y_train = split_data(kaggle_train_df)
X_test = kaggle_test_df


# Split the features into groups


In [13]:
for col in X_train:
    nan_rate = nan_ratio_col(X_train, col)
    print(f"{col} has a nan rate of {nan_rate}")

# find cols to remove
nan_cols_to_remove = [column for column in X_train.columns if nan_ratio_col(X_train, column) > 0.3]
print(f"There are {len(nan_cols_to_remove)} columns that need to be removed")


Id has a nan rate of 0.0
MSSubClass has a nan rate of 0.0
MSZoning has a nan rate of 0.0
LotFrontage has a nan rate of 0.1773972602739726
LotArea has a nan rate of 0.0
Street has a nan rate of 0.0
Alley has a nan rate of 0.9376712328767123
LotShape has a nan rate of 0.0
LandContour has a nan rate of 0.0
Utilities has a nan rate of 0.0
LotConfig has a nan rate of 0.0
LandSlope has a nan rate of 0.0
Neighborhood has a nan rate of 0.0
Condition1 has a nan rate of 0.0
Condition2 has a nan rate of 0.0
BldgType has a nan rate of 0.0
HouseStyle has a nan rate of 0.0
OverallQual has a nan rate of 0.0
OverallCond has a nan rate of 0.0
YearBuilt has a nan rate of 0.0
YearRemodAdd has a nan rate of 0.0
RoofStyle has a nan rate of 0.0
RoofMatl has a nan rate of 0.0
Exterior1st has a nan rate of 0.0
Exterior2nd has a nan rate of 0.0
MasVnrType has a nan rate of 0.5972602739726027
MasVnrArea has a nan rate of 0.005479452054794521
ExterQual has a nan rate of 0.0
ExterCond has a nan rate of 0.0
Founda

In [14]:
# drop rows with too many nans!
X_train.drop(labels=nan_cols_to_remove, axis=1, inplace=True)
X_test.drop(labels=nan_cols_to_remove, axis=1, inplace=True)

In [15]:
len(X_train.columns)

74

In [16]:
oe_features = list({
    "Alley","LotShape",
    "LandContour","Utilities",
    "LotConfig","LandSlope",
    "HouseStyle","OverallQual",
    "OverallCond","ExterQual",
    "ExterCond","BsmtQual",
    "BsmtCond","BsmtExposure",
    "BsmtFinType1","BsmtFinType2",
    "HeatingQC","KitchenQual",
    "Functional","FireplaceQu",
    "GarageFinish","GarageQual",
    "GarageCond","PavedDrive",
    "PoolQC","Fence"} - set(nan_cols_to_remove))

In [17]:
drop_features = []

In [18]:
numerical_features = X_train.select_dtypes(include=np.number).columns

In [19]:
ohe_features = list(set(X_train.columns) - set(oe_features) - set(numerical_features) - set(drop_features))



### clarify dtypes



In [20]:
# Feature groups are: ohe_features oe_features numerical_features drop_features  

In [21]:
X_train[ohe_features].dtypes

Street           object
RoofStyle        object
SaleCondition    object
Electrical       object
GarageType       object
Foundation       object
Heating          object
CentralAir       object
MSZoning         object
Exterior1st      object
RoofMatl         object
Condition1       object
BldgType         object
Exterior2nd      object
Neighborhood     object
Condition2       object
SaleType         object
dtype: object

In [22]:
X_train[numerical_features].dtypes

Id                 int64
MSSubClass         int64
LotFrontage      float64
LotArea            int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
BsmtFinSF1         int64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageYrBlt      float64
GarageCars         int64
GarageArea         int64
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
MiscVal            int64
MoSold             int64
YrSold             int64
dtype: object

In [23]:
X_train[oe_features].dtypes

ExterQual       object
LotConfig       object
BsmtQual        object
OverallQual      int64
ExterCond       object
Utilities       object
HeatingQC       object
GarageCond      object
Functional      object
BsmtCond        object
BsmtFinType2    object
LotShape        object
OverallCond      int64
BsmtFinType1    object
GarageFinish    object
HouseStyle      object
GarageQual      object
PavedDrive      object
BsmtExposure    object
LandSlope       object
KitchenQual     object
LandContour     object
dtype: object

## Feature engineering

In [24]:
### Numerical features

### Consider possible new features

In [25]:
# feature 1:

In [26]:
# feature 2:

In [27]:
# feature 3:

### Create new features

# Brainstorm feature engineering

# Setup pipeline

## Enter the value orders for each ordinal feature

In [28]:
categories = {
    'LotShape': ['IR3', 'IR2', 'IR1', 'Reg'],
    'LandSlope': ['Sev', 'Mod', 'Gtl'],
    'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtQual': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtCond': ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtExposure': ['NA', 'No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'BsmtFinType2': ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'Functional': ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
    'GarageFinish': ['NA', 'Unf', 'RFn', 'Fin'],
    'Utilities': ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
    'LandContour': ['Low', 'HLS', 'Bnk', 'Lvl'],
    'PavedDrive': ['N', 'P', 'Y'],
    'HouseStyle': ['1Story','1.5Fin','1.5Unf','2Story','2.5Fin','2.5Unf','SFoyer','SLvl'], 
    'LotConfig': ['Inside','Corner','CulDSac','FR2','FR3'], 
}

## Create custom transformers

### Ordinal Encoder

In [29]:
!!!

class PersonalisedOrdinalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, order_info):
        """
        order_info: A dictionary where keys are column names and values are lists specifying the order
        """
        # get the encoder details
        self.order_info = order_info
        self.encoders = {}
        
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        # create the encoders
        for col in self.order_info.keys():
            self.encoders[col] = OrdinalEncoder(categories=[self.order_info[col]])
        
        return self
    
    def transform(self, X):
        # encode the data in the correct way
        X_encoded = pd.DataFrame(X.copy())
        for col, encoder in self.encoders.items():
            # check we have all of the possible feature values 

            unique_vals = pd.unique(X_encoded[col])
                
            unexpected_vals = [val for val in unique_vals if val not in self.order_info[col]]
            if unexpected_vals:
                raise Exception(f"That feature value is new...\nLook:{unexpected_vals}")
                
            X_encoded[col] = encoder.fit_transform(X_encoded[[col]])
        
        return X_encoded

### Impute

In [30]:
!!!

class DF_Imputer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        missing_values=np.nan,
        strategy='mean',
        fill_value=None,
        copy=True,
        add_indicator=False,
        keep_empty_features=False):


        self.missing_values =missing_values
        self.strategy = strategy
        self.fill_value = fill_value
        self.copy = copy
        self.add_indicator = add_indicator
        self.keep_empty_features = keep_empty_features
        
        # get the details
        self.columns = None
        self.simple_imputer = SimpleImputer(
            missing_values=self.missing_values,
            strategy=self.strategy,
            fill_value=self.fill_value,
            copy=self.copy,
            add_indicator=add_indicator,
            keep_empty_features=self.keep_empty_features,
        )

    def fit(self, X, y=None):
        self.columns = list(X.columns)
        # fit the imputer
        self.simple_imputer.fit(X)        
        
        return self
    
    def transform(self, X):
        
        # impute and return as pddf
        X_transformed = self.simple_imputer.transform(X)
        
        return pd.DataFrame(columns=self.columns, data=X_transformed, index=X.index)

In [31]:
from sklearn.decomposition import PCA

## Implement ETL pipeline

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

In [33]:
numerical_pipe = Pipeline([
    ('numerical_impute', DF_Imputer(strategy="median")),
    # ('scale', StandardScaler()),
])

one_hot_pipe = Pipeline([
    ('one_hot_impute', DF_Imputer(strategy="most_frequent")),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('one_hot_PCA', PCA(n_components=11)),
    
])

ordinal_pipe = Pipeline([
    ('ordinal_impute', DF_Imputer(strategy="most_frequent")),
    ('ordinal_encoder', PersonalisedOrdinalEncoder(categories)),
    
])

preproc_pipe = ColumnTransformer([
    ('numerical_transformer', numerical_pipe, numerical_features),
    ('one_hot_transformer', one_hot_pipe, ohe_features),
    ('ordinal_transformer', ordinal_pipe, oe_features),
])   

regression_pipeline = Pipeline([
    ('preproc_pipe', preproc_pipe),
    ('regression', GradientBoostingRegressor()),
])

# regression_pipeline.fit(X_train, y_train);
# regression_pipeline.score(X_test, y_test)

In [34]:
#Hp tuning

In [35]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor


# Model building

## Find the best model

In [36]:
params = {

    'regression':[LinearRegression(), Ridge(), Lasso(), ElasticNet(), DecisionTreeRegressor(), RandomForestRegressor(), GradientBoostingRegressor(), AdaBoostRegressor(), SVR(), KNeighborsRegressor(), ExtraTreesRegressor()]
}

model_gridsearch = GridSearchCV(
    regression_pipeline,
    params,
    cv=5,
    scoring="r2"
)

In [37]:
model_gridsearch.fit(X_train, y_train);

In [38]:
pd.DataFrame(model_gridsearch.cv_results_)[["param_regression","mean_test_score","std_test_score","rank_test_score"]].sort_values(by=["rank_test_score"])

,param_regression,mean_test_score,std_test_score,rank_test_score
6,GradientBoostingRegressor(),0.875760,0.022009,1
10,ExtraTreesRegressor(),0.863666,0.032876,2
5,RandomForestRegressor(),0.857352,0.021478,3
1,Ridge(),0.809528,0.070463,4
2,Lasso(),0.809429,0.070479,5
0,LinearRegression(),0.809408,0.070484,6
7,AdaBoostRegressor(),0.805481,0.027401,7
3,ElasticNet(),0.803322,0.075774,8
4,DecisionTreeRegressor(),0.690615,0.077241,9
9,KNeighborsRegressor(),0.630025,0.050504,10


In [39]:
model = model_gridsearch.best_estimator_

## hyperparameter tuning on best model

In [40]:
model

Pipeline(steps=[('preproc_pipe',
                 ColumnTransformer(transformers=[('numerical_transformer',
                                                  Pipeline(steps=[('numerical_impute',
                                                                   DF_Imputer(strategy='median'))]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       '...
                                                                                                                        'AllPub']}))]),
                                                  ['ExterQual', 'LotConfig',
                                                   'BsmtQual', 'OverallQual',
                                                   'ExterCond', 'Utilities',
                                                   'HeatingQC', 'GarageCond',
                                                   'Functional', 'BsmtCond',
                                                   'BsmtFinType2', 'LotShape',
                                                   'OverallCond',
                                                   'BsmtFinType1',
                                                   'GarageFinish', 'HouseStyle',
                                                   'GarageQual', 'PavedDrive',
                                                   'BsmtExposure', 'LandSlope',
                                                   'KitchenQual',
                                                   'LandContour'])])),
                ('regression', GradientBoostingRegressor())])

### Stage 1: Random search

First we will do a random parameter search to get a rough idea of what parameters we should focus on and how much of an impact they have

In [41]:
# GradientBoostingRegressor()
    # loss='squared_error',
    # learning_rate=0.1,
    # n_estimators=100,
    # subsample=1.0,
    # criterion='friedman_mse',
    # min_samples_split=2,
    # min_samples_leaf=1,
    # min_weight_fraction_leaf=0.0,
    # max_depth=3,
    # min_impurity_decrease=0.0,
    # init=None,
    # random_state=None,
    # max_features=None,
    # alpha=0.9,
    # verbose=0,
    # max_leaf_nodes=None,
    # warm_start=False,
    # validation_fraction=0.1,
    # n_iter_no_change=None,
    # tol=0.0001,
    # ccp_alpha=0.0,


In [42]:
model

Pipeline(steps=[('preproc_pipe',
                 ColumnTransformer(transformers=[('numerical_transformer',
                                                  Pipeline(steps=[('numerical_impute',
                                                                   DF_Imputer(strategy='median'))]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       '...
                                                                                                                        'AllPub']}))]),
                                                  ['ExterQual', 'LotConfig',
                                                   'BsmtQual', 'OverallQual',
                                                   'ExterCond', 'Utilities',
                                                   'HeatingQC', 'GarageCond',
                                                   'Functional', 'BsmtCond',
                                                   'BsmtFinType2', 'LotShape',
                                                   'OverallCond',
                                                   'BsmtFinType1',
                                                   'GarageFinish', 'HouseStyle',
                                                   'GarageQual', 'PavedDrive',
                                                   'BsmtExposure', 'LandSlope',
                                                   'KitchenQual',
                                                   'LandContour'])])),
                ('regression', GradientBoostingRegressor())])

In [43]:
param_grid = {
    "regression__learning_rate": [0.01, 0.1, 0.2, 0.05],
    "regression__loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "regression__n_estimators": [10, 50, 100, 200, 500],
    "regression__criterion": ["friedman_mse", "squared_error"],
    "regression__min_samples_split": [0,1,2,3,4],
    "regression__min_samples_leaf": [0,1,2,3,4],
    "regression__max_depth": [2,3,4],
    "regression__max_features": ["sqrt", "log2"],
    "regression__n_iter_no_change": [None, 1, 2],
}

# hyperparam_randomsearch = RandomizedSearchCV(estimator=model, param_distributions=random_params, cv=5, n_iter=2, random_state=42, n_jobs=-1)
hyperparam_gridsearch = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring="r2", n_jobs=-1)

In [ ]:
hyperparam_gridsearch.fit(X_train, y_train);

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

...

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/javonne/anaconda3/envs/mlds4p/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when 

In [ ]:
pd.DataFrame(hyperparam_gridsearch.cv_results_)[["mean_test_score","std_test_score","rank_test_score"]].sort_values(by=["rank_test_score"])


In [ ]:
tuned_model = hyperparam_gridsearch.best_estimator_

# evaluate on test data

In [ ]:
# GSearched GradientBoostingRegressor
# RMSE 
# Tuned GradientBoostingRegressor
# RMSE 13333.324115873318
# GradientBoostingRegressor
# RMSE 14231.964170370431
# Linear Regression
# RMSE 32068.866744798997

In [ ]:
np.sqrt(mean_squared_error(tuned_model.predict(X), y))

# predict on external validation set

In [ ]:
:

In [ ]:
import csv

def save_predictions_to_csv(predictions, filename="predictions.csv"):
    """
    Saves a list of predictions to a CSV file with IDs starting from 1461.
    
    Parameters:
    - predictions: A list (or list-like structure) of float predictions of shape (1459, 1).
    - filename: The name of the CSV file to save the predictions to.
    """
    # Ensure the predictions are in a 2D list form if they're not
    if predictions.ndim > 1:
        predictions = predictions.flatten().tolist()
    
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(['Id', 'SalePrice'])
        
        # Write each prediction with its corresponding ID
        for i, prediction in enumerate(predictions, start=1461):
            writer.writerow([i, prediction])

save_predictions_to_csv(tuned_model.predict(X_test))